In [ ]:
from pathlib import Path
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Налаштування аугментацій
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    brightness_range=(0.8, 1.2),
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='reflect'
)

# Типи оброблених даних
types = ["color", "grayscale", "segmented"]

for t in types:
    input_root  = Path("../../data/processed") / t
    output_root = Path("../../data/augmented") / t

    for cls_dir in input_root.iterdir():
        if not cls_dir.is_dir():
            continue

        out_cls_dir = output_root / cls_dir.name
        out_cls_dir.mkdir(parents=True, exist_ok=True)

        # Проходимо всі зображення даного класу
        for img_path in cls_dir.glob("*.*"):
            img = cv2.imread(str(img_path), cv2.IMREAD_UNCHANGED)
            if img is None:
                continue

            # Якщо grayscale — перетворимо в трьохканальне RGB
            if img.ndim == 2:
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
            else:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            # Готуємо батч розміром 1
            batch = img.reshape((1, *img.shape))

            # Генеруємо по 5 аугментованих варіантів
            i = 0
            for aug in datagen.flow(batch, batch_size=1,
                                     save_to_dir=str(out_cls_dir),
                                     save_prefix="aug", save_format="png"):
                i += 1
                if i >= 5:
                    break

print("✅ Аугментації створені для всіх типів і класів.")
